# Installing Libraries

In [1]:
#!pip install pycaret
import pandas as pd
from pycaret.regression import *

# Loading Dataset and Model

In [2]:
test = pd.read_csv("./Dataset/Test Dataset.csv")
check = pd.read_csv("./Dataset/Actual Results of Test Dataset.csv")
sih_model = load_model('./Model/sih_motor_xgboost_efficiency_regressor')

Transformation Pipeline and Model Sucessfully Loaded
[00:21:49] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


# Prediction

In [3]:
def relia(pv1, pi1, t1, a1, w1): 
    
    sd_v,sd_i,sd_t,sd_w,sd_age,a_v,a_i,a_t,a_w,a_age = 14.450315,1.184078,14.724061,11.850687,3,425.041214,10.004947,55.033790,99.990990,5        
    
    reliabilty = 5
    if pv1 > a_v + 1.2*sd_v:
        reliabilty -=1
    if pi1 > a_i + 1.2*sd_i:
        reliabilty -=1
    if t1 > a_t + 1.2*sd_t:
        reliabilty -=1
    if w1 > a_w + 1.2*sd_w:
        reliabilty -=1
    if a1 > a_age + sd_age:
        reliabilty -=1

    if reliabilty == 5:
        return 'Highly Reliable'
    if reliabilty == 3 or reliabilty == 4:
        return 'Moderately Reliable'
    if reliabilty == 1 or reliabilty == 2:
        return 'Less Reliable'
    if reliabilty == 0:
        return 'Not Reliable'

pred_new = predict_model(sih_model,data=test)
trained = test

r,c=[],[]

for i in range(pred_new.count()[1]):
    r.append(relia(pred_new['3 Phase Voltage in Volts'][i],pred_new['3 Phase Current in Amps'][i],pred_new['Temperature in Celcius'][i],pred_new['Motor Age in Yrs'][i],pred_new['Angular Velocity in rad/sec'][i]))
    e1 = pred_new['Label'][i]
    ctr = round(50 - (3500/e1),0)
    if ctr < 0 : ctr = 0
    c.append(abs(ctr))

trained['Reliability_Check'] = r
trained['Life Cycle in Yrs_Check'] = c
trained['Predicted Effeciency_Check'] = pred_new['Label']

test.to_csv('./Dataset/Predicted Results of Test Dataset.csv',index=False)